In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
from sklearn.metrics.pairwise import cosine_similarity
import operator
import gc
#import torch
import zipfile

In [ ]:
#archivo = zipfile.ZipFile('ml10m.zip')
archivo = zipfile.ZipFile('ml-1m.zip')

In [ ]:
archivo.extractall('')

In [3]:
movie_col = ['movie_id', 'titulo', 'genero']
movie = pd.read_csv('movies.dat', sep='::', names=movie_col)

/home/mich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [4]:
movie.head()

,movie_id,titulo,genero
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


# Preprocesamiento de los datos de movie

Vamos a separar el año del titulo para que sea más fácil hacer la búsqueda de los titulos sin necesidad de poner el año de estreno.

In [5]:
nombres = movie['titulo'].tolist() #Sacamos los titulos de las películas en una lista para hacer split entre los nombres y el año

In [ ]:
nombres[0][-6:]

In [6]:
#A cada elemento de nombre se le divide en 2 partes: el nombre de la película dentro de la lista name 
#y dentro de la lista year va el año de cada película
name = []
year = []
for i in nombres:
    name.append(i[:-7])
    year.append(i[-6:])

In [ ]:
name[:10]

In [ ]:
year[:10]

In [7]:
#Convertimos ambas listas con dataframes para poder agregarlas al df final
name = pd.DataFrame(name, columns=['titulo'])
years = pd.DataFrame(year, columns=['year'])

In [8]:
movie.drop(movie.columns[1], axis=1, inplace=True) # Quitamos la columna de titulo del df original

movie = pd.merge(movie, name, left_on=None, right_on=None, left_index=True, right_index=True)
movie = pd.merge(movie, years, left_on=None, right_on=None, left_index=True, right_index=True)

In [ ]:
movie.head()

In [9]:
cols = ['movie_id', 'titulo', 'year', 'genero'] #Reordenamos las columnas solo para que se vea más bonito
movie = movie[cols]
movie.head()

,movie_id,titulo,year,genero
0,1,Toy Story,(1995),Animation|Children's|Comedy
1,2,Jumanji,(1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men,(1995),Comedy|Romance
3,4,Waiting to Exhale,(1995),Comedy|Drama
4,5,Father of the Bride Part II,(1995),Comedy


In [10]:
rating_col = ['user_id', 'movie_id', 'rating']
ratings = pd.read_csv('ratings.dat', sep='::', names=rating_col, usecols=range(3))

/home/mich/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  


In [ ]:
ratings.head()

In [ ]:
usuario_col = ['user_id']
usuarios = pd.read_csv('users.dat', sep='::', names=usuario_col, usecols=range(1))

In [ ]:
usuarios.head()

In [11]:
movie_rating = pd.merge(movie, ratings) # Juntamos los df de ratings y películas

In [12]:
movie_rating.head()

,movie_id,titulo,year,genero,user_id,rating
0,1,Toy Story,(1995),Animation|Children's|Comedy,1,5
1,1,Toy Story,(1995),Animation|Children's|Comedy,6,4
2,1,Toy Story,(1995),Animation|Children's|Comedy,8,4
3,1,Toy Story,(1995),Animation|Children's|Comedy,9,5
4,1,Toy Story,(1995),Animation|Children's|Comedy,10,5


In [ ]:
movie_rating.shape

Creamos un df que tiene como filas 'user_id' y columnas los titulos de películas, por lo tanto, los datos corresponden al rating que el usuario x le asigna a la película y

In [13]:
datos_completos = movie_rating.pivot_table(index=['user_id'],columns=['titulo'],values='rating')

In [14]:
datos_completos.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
user_id,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
datos_completos.shape

In [15]:
datos_completos = datos_completos.fillna(0);

In [16]:
datos_norm = datos_completos.apply(lambda x: (x-np.mean(x))/np.std(x), axis=1)

In [17]:
datos_norm.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
user_id,,,,,,,,,,,,,,,,,,,,,
1,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,...,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587,-0.119587
2,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,...,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261,-0.184261
3,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,...,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216,-0.115216
4,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,...,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627,-0.073627
5,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,...,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218,-0.224218


In [18]:
gc.collect()

44

In [19]:
matrix = datos_norm.values # Creamos una matriz con los valores del df normalizado

In [20]:
matrix.shape

(6040, 3664)

In [21]:
movie_similar = cosine_similarity(matrix.T) #Calculamos la distancia del coseno entre películas

In [22]:
#Creamos un df para hacer más fácil la visualización de la simulitud de las películas
movie_sim_df = pd.DataFrame(movie_similar, index = datos_norm.T.index, columns = datos_norm.T.index)

In [23]:
movie_sim_df.head()

titulo,"$1,000,000 Duck",'Night Mother,'Til There Was You,"'burbs, The",...And Justice for All,1-900,10 Things I Hate About You,101 Dalmatians,12 Angry Men,"13th Warrior, The",...,"Young Poisoner's Handbook, The",Young Sherlock Holmes,Young and Innocent,Your Friends and Neighbors,Zachariah,"Zed & Two Noughts, A",Zero Effect,Zero Kelvin (Kj�rlighetens kj�tere),Zeus and Roxanne,eXistenZ
titulo,,,,,,,,,,,,,,,,,,,,,
"$1,000,000 Duck",1.000000,0.226403,0.287241,0.013680,0.038991,0.515257,-0.102371,-0.019481,-0.091710,-0.108637,...,0.195469,-0.046259,0.504575,0.151727,0.534177,0.342827,-0.044329,0.552773,0.423281,-0.057637
'Night Mother,0.226403,1.000000,0.211111,0.029864,0.069808,0.340435,-0.060055,-0.045779,-0.064600,-0.092479,...,0.124712,-0.046943,0.345423,0.133307,0.352796,0.244651,-0.029017,0.364859,0.262286,-0.037362
'Til There Was You,0.287241,0.211111,1.000000,0.101821,0.079687,0.527723,-0.033479,-0.048342,-0.068866,-0.040252,...,0.160314,-0.055961,0.442899,0.147584,0.469536,0.295178,-0.018119,0.485826,0.364868,-0.042425
"'burbs, The",0.013680,0.029864,0.101821,1.000000,0.111827,0.010899,0.077131,0.089290,0.091483,0.059172,...,0.019160,0.029011,0.009646,0.013183,0.010859,0.007766,-0.001345,0.010550,0.004749,-0.026783
...And Justice for All,0.038991,0.069808,0.079687,0.111827,1.000000,0.045556,-0.015593,0.056237,0.120471,0.040309,...,0.021856,-0.000602,0.055240,0.021200,0.046917,0.043497,-0.015681,0.051674,0.027438,-0.017999


In [24]:
gc.collect()

132

In [ ]:
torch.cuda.empty_cache()

In [25]:
#hacemos el mismo procedimiento para el df de usuario y ver cuales son más similares entre si
usuarios_similar = cosine_similarity(matrix)

In [26]:
usuarios_sim_df = pd.DataFrame(usuarios_similar, index = datos_norm.index, columns = datos_norm.index)

In [ ]:
usuarios_sim_df.head()

In [27]:
# Toma las 5 películas más 'parecidas' a la pelicula que se le de como input. También toma el score de similitud

def top_movies(movie):
    movie_sim = movie_sim_df.sort_values(by = movie, ascending=False).index[1:6]
    valores_sim = movie_sim_df.sort_values(by = movie, ascending=False).loc[:,movie].tolist()[1:6]
    zipped = zip(movie_sim, valores_sim,)
    print('Películas similares a ', movie)
    for mov, sim in zipped:
        print(mov, 'tiene similitud de ', sim)

In [28]:
# Toma los usuarios con mayor similitud dado un usuario 'x'. También toma el score de similitud

def top_users(usuario, n):
    if usuario not in datos_completos.index:
        return('No existe usuario')
    else:
        valores_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).loc[:,usuario].tolist()[1:n+1]
        usuario_sim = usuarios_sim_df.sort_values(by = usuario, ascending=False).index[1:n+1]
        #zipped = zip(usuario_sim, valores_sim,)
        parejas = []
        for v in range(len(valores_sim)):
            parejas.append((usuario_sim[v], valores_sim[v]))
            #print('El usuario ', user, 'tiene similitud de ', sim)
        return parejas

In [29]:
def obtener_promedio(usuario):
    vistos = datos_completos.T[datos_completos.loc[usuario,:]>0].index.tolist()
    
    prom = []
    for i in vistos:
        prom.append(datos_completos.loc[usuario, i])
        
    return np.mean(prom)

In [30]:
def prediccion(peli, usuario, n):
    sim = top_users(usuario, n)
    sim_total = sum([i[1] for i in sim])
    lambdas = 0
    pred = 0
    for i in sim:
        lambdas = (i[1]/sim_total)
        cal_movie = datos_completos.loc[i[0], peli]
        prom = obtener_promedio(usuario)
        prom1 = obtener_promedio(i[0])
        pred += lambdas*(cal_movie-prom1)

    return round(pred + prom)

In [31]:
top_movies('10 Things I Hate About You')

Películas similares a  10 Things I Hate About You
American Pie tiene similitud de  0.40267642925343555
Austin Powers: The Spy Who Shagged Me tiene similitud de  0.3486986601755031
She's All That tiene similitud de  0.34770734191661795
Big Daddy tiene similitud de  0.3415634037040133
Cruel Intentions tiene similitud de  0.3306761084477506


In [32]:
top_movies('Toy Story')

Películas similares a  Toy Story
Toy Story 2 tiene similitud de  0.5109854855905952
Groundhog Day tiene similitud de  0.46347078796835933
Aladdin tiene similitud de  0.44192800314573616
Babe tiene similitud de  0.43115005720082084
Bug's Life, A tiene similitud de  0.39153166680818036


In [44]:
prediccion('Toy Story', 1500, 25)

3.0

# Calcular exactitud

In [34]:
from sklearn.metrics import f1_score

In [45]:
def error(usuario, n):
    vistos = datos_completos.T[datos_completos.loc[usuario,:]>0].index.tolist()
    real =[]
    pred = []
    for i in vistos:
        real.append(datos_completos.loc[usuario, i])
        pred.append(prediccion(i, usuario, n))
    return real, pred

In [55]:
y_true, y_pred = error(3, 40)

In [39]:
zip(y_true, y_pred)

In [53]:
for i, j in zip(y_true, y_pred):
    print(i, j)

3.0 0.0
4.0 3.0
4.0 3.0
3.0 3.0
3.0 1.0
3.0 0.0
5.0 1.0
5.0 2.0
4.0 2.0
5.0 2.0
5.0 2.0
5.0 2.0
4.0 2.0
4.0 3.0
4.0 1.0
4.0 1.0
4.0 1.0
5.0 0.0
1.0 0.0
5.0 2.0
2.0 1.0
4.0 1.0
4.0 1.0
2.0 2.0
4.0 1.0
2.0 0.0
4.0 4.0
4.0 3.0
4.0 3.0
4.0 1.0
4.0 2.0
3.0 3.0
3.0 2.0
5.0 3.0
2.0 1.0
3.0 1.0
5.0 4.0
5.0 4.0
4.0 2.0
4.0 2.0
5.0 3.0
3.0 2.0
5.0 3.0
5.0 4.0
4.0 4.0
4.0 4.0
4.0 1.0
3.0 2.0
5.0 1.0
5.0 1.0
4.0 1.0


In [66]:
f1_score(y_true, y_pred, average='weighted')

0.16119117330189997

In [ ]:
vist